SISTEMA DE RECOMENDACION

In [1]:
# Importamos las librerías
from fastapi import FastAPI
import pandas as pd
import numpy as np
import ast
import re
from sklearn.neighbors import NearestNeighbors

In [6]:
# Cargar los datos del DataFrame
df = pd.read_csv('steam_games.csv')

In [7]:
# Características categóricas
genres_dummies = df['Genres'].str.get_dummies(sep=', ')
specs_dummies = df['Specs'].str.get_dummies(sep=', ')
features_matrix = pd.concat([genres_dummies, specs_dummies], axis=1)

In [8]:
# Entrenar un modelo K-NN
k = 5  # Número de vecinos cercanos a considerar
model = NearestNeighbors(n_neighbors=k, metric='cosine')
model.fit(features_matrix)

NearestNeighbors(metric='cosine')

In [9]:
def get_recommendations(user_id, num_recommendations=5):
    # Encuentra el índice del usuario en el DataFrame
    user_index = df[df['User_Id'] == user_id].index[0]
    
    # Encuentra los vecinos más cercanos (usuarios similares)
    distances, indices = model.kneighbors([features_matrix.iloc[user_index]], n_neighbors=num_recommendations + 1)
    
    # Excluye al propio usuario de las recomendaciones
    recommended_user_indices = indices[0][1:]
    
    # Obtiene juegos recomendados para el usuario
    recommended_games = df.iloc[recommended_user_indices]['App_Name'].tolist()
    
    return recommended_games



In [11]:
get_recommendations(761140)

C:\Users\COMPUTER\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Wooden Battles',
 'Trivia Vault Super Heroes Trivia',
 'Trivia Vault Mini Mixed Trivia',
 'Crankies Workshop Whirlbot Assembly',
 'Trivia Vault Mini Mixed Trivia']

In [ ]:
@app.get("/recommendations/{user_id}")
def get_user_recommendations(user_id: int, num_recommendations: int = 5):
    
    '''Obtiene 5 juegos recomendados para el usuario basados en sus interacciones previas.'''
    
    recommended_games = get_recommendations(user_id, num_recommendations)
    return {"recommended_games": recommended_games}